In [20]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd



# Laod packages

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed

In [21]:
# Read config parser .ini file with your connection information

Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
dotenv.load_dotenv(Path)

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 


# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZJBb%2BIwEIX%2FSuQ9xw4u3S0WUFFQtayaFpGwor2Z2ICFY2c9Dmn769cJIHUP7WFvkfNmvjfzZnj7WuroKB0oa0aohxMUSVNYocxuhFb5fXyDIvDcCK6tkSP0JgHdjofAS12xSe33Zin%2F1BJ8FBoZYO2PEaqdYZaDAmZ4KYH5gmWT9IFRnDAOIJ0POHQuEaACa%2B99xQhpmgY3V9i6HaFJkpBkQIKqlXxDHxDV14zKWW8Lqy8lr2GmTxA9kvRbRFAEwuJceKfMaQVfUTYnEbCfeb6IF09ZjqLJZbqpNVCX0mXSHVUhV8uHkwEIDpQ5qrDU4FLgGmLJwccU8wYwGNtsNT%2FIwpZV7UN3HL7IVgqi7U6Fnc1nI1QdlNi8L%2Fv1NN2sD8eXAb9e7Kdpf5Xf2X76K6UDvXve0LVap4%2FL7XOBot%2BXhGmb8ByglnPT5urDU0Kv44TGyfecUta7YpTiG%2FrjBUWzkKsy3HeVF%2FM8hI4Pttgr0XlToiJZ9oTbWCg6nQbrCG78vwMPyccu52t7DAHMZwurVfEW3VtXcv95Pj3c616UiLedlMmSKz0RwkmAkJPWtpk6yX04au9qicj4RP33rMd%2FAQ%3D%3D&RelayState=ver%3A1-hint%3A759822674653786-ETMsDgAAAZTdUysLABRBRVMvQ0JD

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [22]:
def get_data(session: Session) -> pd.DataFrame:
    # Load data from Snowflake
    """
    This function loads data from the table PURCHASES in the Snowflake database

    Args:
        session (Session): snowflake session object

    Returns:
        pd.DataFrame: returns the data from the table PURCHASES
    """
    with open("/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/data/purchasing_anomaly.sql", "r") as file:
        query = file.read()
    df = pd.read_sql(query, session.connection)
    return df

In [23]:
df = get_data(session)

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_44396/3770731770.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


In [11]:
df.head()

,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor ID,Vendor NM,Plant Desc,Purchase Doc ID,Purchase Doc Item ID,...,Cost Center Category Desc,Work Order No,Network ID,Project ID,Currency Desc,Long Text,Purchase Doc Item Ct,PO Qty,PO Item Net Order Value USD,User Desc
0,2025-02-04,2025-02,2025 QTR-1,2025,2025-02-03,8010000984,GRAINGER INC,Victoria (US) MFG,2000136963,10,...,Maintenance,,,NaN,US Dollar,None,1,1.0,360.670,"Olascuaga, Randy"
1,2023-11-16,2023-11,2023 QTR-4,2023,2023-11-23,8010000984,GRAINGER INC,Victoria (US) MFG,2000091171,100,...,Maintenance,,,NaN,US Dollar,None,1,6.0,140.040,"Girdy, Brenton"
2,2023-11-16,2023-11,2023 QTR-4,2023,2023-11-16,8010000984,GRAINGER INC,Victoria (US) MFG,2000091165,220,...,Maintenance,,,NaN,US Dollar,None,1,30.0,384.300,"Girdy, Brenton"
3,2024-08-15,2024-08,2024 QTR-3,2024,2024-08-21,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000118933,120,...,Maintenance,,,NaN,Canadian Dollar,None,1,100.0,18.797,"Louwerse, James"
4,2024-12-17,2024-12,2024 QTR-4,2024,2024-12-18,8010000984,GRAINGER INC,Seaford (US) MFG,2000132043,20,...,Direct Production,,,NaN,US Dollar,None,1,1.0,16.320,"Ennis, Jeffery"


In [26]:
# Define stopwords manually (Scikit-learn's stopword list is English only)
stopwords = set([
    'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
    'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
    'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
    'any', 'am', 'all', 'after', 'again', 'above', 'by',
    'should', 'did', 'each', 'had', 'from', 'her', 'him',
    'but', 'by', 'below', 'between', 'at', 'such', 'some',
    'same', 'out', 'with', 'and', 'is', 'so', 'staples'
])

def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Filter dataset to keep only Purchase Orders starting with '55' or '20'."""
    df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
    return df

def get_date_features(df: pd.DataFrame, column_name: str):
    """Extract year and month from a date column."""
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    df[f'{column_name} Year'] = df[column_name].dt.year
    df[f'{column_name} Month'] = df[column_name].dt.month
    return df

def clean_text(text_series: pd.Series) -> pd.Series:
    """Cleans text: removes stopwords, digits, and short words."""
    text_series_clean = text_series.str.lower()
    text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
    text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
    return text_series_clean

def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
    """Returns unique words from the clean text column."""
    all_words = df[clean_text_col].str.split().explode().unique()
    return list(set(all_words))

def first_element(text: str) -> str:
    """Extracts the first meaningful part of a text (for vendor-specific processing)."""
    return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
    """Extracts the relevant short text for each vendor."""
    df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

    for i in df.index:
        long_text = df.loc[i, 'Long Text']  # Use the correct column name

        if pd.isna(long_text):  # If Long Text is NaN, use Short Text
            df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
        elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
            df.loc[i, 'text_clean'] = first_element(long_text)
        elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
        elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            product_text = str(long_text).split(', ')[0]
            df.loc[i, 'text_clean'] = product_text.split('---')[0]
    
    return df


def extract_nouns_tfidf(text_series: pd.Series) -> list:
    """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
    vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
    X = vectorizer.fit_transform(text_series)

    # Convert TF-IDF matrix back to text
    feature_names = vectorizer.get_feature_names_out()

    # Extract top words per row
    word_lists = []
    for row in X.toarray():
        top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
        top_words = [feature_names[i] for i in top_indices if row[i] > 0]
        word_lists.append(" ".join(top_words))

    return word_lists

def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """Extracts top words using TF-IDF as a proxy for important words."""
    df['noun_words'] = extract_nouns_tfidf(df[text_col])
    return df

def get_noun_words_reduced(df: pd.DataFrame):
    """Filters clean text to retain only top 5 extracted nouns."""
    df['text_clean'] = df['text_clean'].str.split()
    df['noun_words'] = df['noun_words'].str.split()
    
    df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
    
    df['text_clean'] = df['text_clean'].str.join(' ')
    df['noun_words'] = df['noun_words'].str.join(' ')
    
    return df

def text_parallel_clean(array):
    """Parallel processing wrapper for extract_nouns_tfidf."""
    result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
    return result

def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
    """Converts lists in a Pandas series to a list of joined strings."""
    return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Formats column names for consistency."""
    df.columns = df.columns.str.replace('_', ' ').str.title()
    return df

In [27]:
df = process_data(df)
df = get_short_text_by_vendor(df)
df['text_clean'] = clean_text(df['text_clean'])
df = extract_noun_words(df, "text_clean")
df = get_noun_words_reduced(df)
df = clean_cols(df)

In [30]:
# Ensure 'Purchase Doc Date' is in datetime format
df['Purchase Doc Date'] = pd.to_datetime(df['Purchase Doc Date'], errors='coerce')

# Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
df = df.sort_values(by='Purchase Doc Date', ascending=True).reset_index(drop=True)

In [38]:
df[['Purchase Doc Date','Purchase Document Item Short Text', 'Noun Words Reduced', 'Noun Words',"Plant Desc"]].tail(50)

,Purchase Doc Date,Purchase Document Item Short Text,Noun Words Reduced,Noun Words,Plant Desc
949,2025-01-15,"SHOKZ New OpenRun Pro 2 - Open-Ear, Bone",shokz new openrun openear,bone new openear shokz openrun,Victoria (US) MFG
950,2025-01-17,Gorilla Rubber Cement with Brush Applica,rubber cement brush,cement brush rubber,Seaford (US) MFG
951,2025-01-20,TK116027261T Elbow Application Air Cutti,tkt elbow application air,elbow application air tkt,Camden Polymer (US) MFG
952,2025-01-20,Carhartt Men's WP Waterproof Insulated G,mens waterproof insulated,waterproof mens insulated,Victoria (US) MFG
953,2025-01-20,KNIPEX - 71 01 200 Tools - CoBolt Compac,tools compact bolt,bolt cutter compact tools,Victoria (US) MFG
954,2025-01-20,Case of 500ML Water,case water,case water,Kingston (CA) MFG
955,2025-01-21,Govee Indoor Hygrometer Thermometer 3 Pa,indoor thermometer,indoor thermometer,Camden Polymer (US) MFG
956,2025-01-21,TK116048661T Dish Soap Washing Type Hand,tkt dish soap type,soap dish hand type tkt,Camden Polymer (US) MFG
957,2025-01-21,"Swingline Desktop Stapler, 20 Sheet Capa",swingline desktop stapler sheet,sheet stapler desktop swingline,Longview (US) MFG
958,2025-01-22,[Apple MFi Certified] Lightning to HDMI,apple mfi certified lightning hdmi,mfi hdmi lightning apple certified,Victoria (US) MFG


In [39]:
def get_last_week_data_for_each_plant(df: pd.DataFrame) -> pd.DataFrame:
    """Filters data to keep only the last week of data for each plant."""
    df = df.groupby('Plant Desc').apply(lambda x: x.nlargest(7, 'Purchase Doc Date')).reset_index(drop=True)
    return df

In [40]:
get_last_week_data_for_each_plant(df)

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_44396/1210642113.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Plant Desc').apply(lambda x: x.nlargest(7, 'Purchase Doc Date')).reset_index(drop=True)


,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor Id,Vendor Nm,Plant Desc,Purchase Doc Id,Purchase Doc Item Id,...,Project Id,Currency Desc,Long Text,Purchase Doc Item Ct,Po Qty,Po Item Net Order Value Usd,User Desc,Text Clean,Noun Words,Noun Words Reduced
0,2025-01-27,2025-01,2025 QTR-1,2025,2025-01-28,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000136118,10,...,NaN,US Dollar,None,1,1.0,39.580,"Lipsey, Cynthia",lcl compatible toner cartridge replaceme,replaceme compatible cartridge toner,compatible toner cartridge replaceme
1,2025-01-21,2025-01,2025 QTR-1,2025,2025-01-24,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000135578,30,...,NaN,US Dollar,None,1,1.0,34.980,"Catoe, Scottie",govee indoor hygrometer thermometer,indoor thermometer,indoor thermometer
2,2025-01-21,2025-01,2025 QTR-1,2025,2025-01-27,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000135541,10,...,NaN,US Dollar,None,1,1.0,56.540,"Brown, Obed",tkt dish soap washing type hand,soap dish hand type tkt,tkt dish soap type
3,2025-01-20,2025-01,2025 QTR-1,2025,2025-01-23,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000135495,10,...,NaN,US Dollar,None,1,2.0,97.920,"Grizzle, Darren",tkt elbow application air cutti,elbow application air tkt,tkt elbow application air
4,2025-01-06,2025-01,2025 QTR-1,2025,2025-01-10,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,5501809041,90,...,NaN,US Dollar,OLIGHT Opry Multi-functional EDC Ti Pry BarSet...,1,1.0,49.950,"Phillips, Tracy",olight opry multifunctional edc pry barset,edc pry,edc pry
5,2025-01-03,2025-01,2025 QTR-1,2025,2025-01-06,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000133483,60,...,NaN,US Dollar,None,1,1.0,12.300,"Lipsey, Cynthia",global printed products horiz,printed products,printed products
6,2024-12-30,2024-12,2024 QTR-4,2024,2025-01-10,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000133026,20,...,NaN,US Dollar,None,1,2.0,359.720,"Gainey, Frank",tkt collection cup capacity,cup capacity tkt,tkt cup capacity
7,2025-01-14,2025-01,2025 QTR-1,2025,2025-01-18,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000134830,100,...,NaN,US Dollar,None,1,2.0,118.360,"Askew, Jeremy",hersheys milk chocolate whole almo,chocolate,chocolate
8,2025-01-13,2025-01,2025 QTR-1,2025,2025-01-20,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000134622,20,...,NaN,US Dollar,None,1,1.0,30.990,"Williams, Susan",carpet chair mat,chair,chair
9,2025-01-09,2025-01,2025 QTR-1,2025,2025-01-13,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000134237,10,...,NaN,US Dollar,None,1,1.0,9.150,"Purdion Shute, Lexus",swingline pack standard stap,swingline standard pack,swingline pack standard


In [47]:
df.shape

(999, 44)